In [1]:
from invisibleroads_macros_disk import make_folder
from os import environ

input_folder = environ.get('CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = make_folder(environ.get('CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output'))
debug_folder = make_folder(environ.get('CROSSCOMPUTE_DEBUG_FOLDER', 'tests/standard/debug'))

In [2]:
from os.path import join, exists

source_path = join(input_folder, 'url.txt')
target_path = join(output_folder, 'geometries.json')
if 'JPY_PARENT_PID' not in environ and exists(target_path):
    exit()

In [3]:
from invisibleroads_macros_disk import unarchive_safely
from os.path import basename, exists, splitext
from urllib.request import urlretrieve as download

source_url = open(source_path, 'rt').read()
archive_name = basename(source_url)
archive_path = join(debug_folder, archive_name)
archive_folder = splitext(archive_path)[0]
if not exists(archive_folder):
    download(source_url, archive_path)
    unarchive_safely(archive_path, archive_folder)

In [8]:
from glob import glob
from pandas import read_excel

spreadsheet_paths = glob(join(archive_folder, '**/*.xlsx'), recursive=True)
if not spreadsheet_paths:
    raise Exception('could not find spreadsheet')
old_table = read_excel(spreadsheet_paths[0])
new_table = old_table[['NAME', 'LON', 'LAT']].rename({
    'NAME': 'name', 'LON': 'longitude', 'LAT': 'latitude'}, axis=1)
new_table.to_csv(join(output_folder, 'geometries.csv'), index=False)

In [9]:
import json

json.dump({
    'longitude': new_table.longitude.mean(),
    'latitude': new_table.latitude.mean(),
    'zoom': 5,
}, open(join(output_folder, 'geometries.json'), 'wt'))